In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# !pip install datasets
# !pip install evaluate

In [ ]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import torch

In [ ]:
ds = load_dataset("EdinburghNLP/xsum", trust_remote_code=True, streaming=False)

In [ ]:
sample = ds['train'][0]

In [ ]:
print(sample.keys())
print('')
print(sample['document'])
print('')
print(sample['summary'])

In [ ]:
summarizer = pipeline('summarization', model='facebook/bart-large-cnn')

In [ ]:
summary = summarizer(sample['document'])

In [ ]:
print(sample['summary'])
print('')
print(summary[0]['summary_text'])

del summarizer

In [ ]:
# Manual pipeline

# tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
# model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')
# tokenized_text = tokenizer(sample['document'], return_tensors='pt')
# output = model.generate(**tokenized_text, num_beams=4, max_length=150, early_stopping=True)
# decoded = tokenizer.decode(output[0], skip_special_tokens=True)
# print(decoded)

# del tokenizer, model

In [ ]:
sample2 = ds['train'][2]
print(sample2['document'])
print('')
print(sample2['summary'])

In [ ]:
bertscore = evaluate.load('bertscore')

In [ ]:
# Compare summarizer to reference
# predictions = [summary[0]['summary_text']]
# references = [sample['summary']]

# Compare full text to summary
references = [sample['document']]
predictions = [sample['summary']]

# Sanity check
# references = [sample['summary']]
# predictions = [sample['summary']]

# Sanity check
# references = [sample['document']]
# predictions = [sample2['document']]

# Sanity check
# references = [sample['summary']]
# predictions = [sample2['summary']]

# references = ['I hate you']
# predictions = ['I love you']

results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
results

In [ ]:
bleurt = evaluate.load("bleurt", module_type="metric")

In [ ]:
# Compare summarizer to reference
# predictions = [summary[0]['summary_text']]
# references = [sample['summary']]

# Compare full text to summary
references = [sample['document']]
predictions = [sample['summary']]

# Sanity check
references = [sample['summary']]
predictions = [sample['summary']]

# Sanity check
# references = [sample['document']]
# predictions = [sample2['document']]

# Sanity check
# references = [sample['summary']]
# predictions = [sample2['summary']]

# references = ['I hate you']
# predictions = ['I love you']

results = bleurt.compute(predictions=predictions, references=references)
results

In [ ]:
del bertscore, bleurt

In [ ]:
nli = pipeline("text-classification", model="FacebookAI/roberta-large-mnli")

In [ ]:
reference = sample['summary']
prediction = summary[0]['summary_text']

# Compare full text to summary
reference = sample['document']
prediction = sample['summary']

# Sanity check
reference = sample['summary']
prediction = sample['summary']

# Sanity check
reference = sample['document']
prediction = sample2['document']

# Sanity check
reference = sample['summary']
prediction = sample2['summary']

# reference = 'I hate you'
# prediction = 'I love you'

# reference = 'I like you'
# prediction = 'I love you'

# reference = 'I love you'
# prediction = 'I want to be with you'

print(f"{reference} </s> {prediction}")
results = nli(f"{reference} </s> {prediction}", truncation=True)
results

In [ ]:
del nli

In [ ]:
bleu = evaluate.load("bleu")

predictions = [summary[0]['summary_text']]
# predictions = [sample['summary']]
references = [sample['summary']]
results = bleu.compute(predictions=predictions, references=references)
results

In [ ]:
rouge = evaluate.load('rouge')

predictions = [summary[0]['summary_text']]
# predictions = [sample['summary']]
references = [sample['summary']]
results = rouge.compute(predictions=predictions, references=references)
results